In [1]:
import os
import itertools
import pandas as pd
pd.set_option('display.max_colwidth', None)
import random

import yaml
yaml.Dumper.ignore_aliases = lambda *args : True

In [7]:
models =['Input', 'DCUNet', 'DPTNet', 'RegressionFCNN', 'SMoLnet', 'WaveUNet']
snrs = ['0', '-5', '-10', '-15', '-20', '-25', '-30']
meta_data_dir = 'meta_data_for_yaml'
os.listdir(meta_data_dir)

['website_meta_data_snr_-15dB.csv',
 'website_meta_data_snr_-5dB.csv',
 'website_meta_data_snr_-30dB.csv',
 'website_meta_data_snr_-10dB.csv',
 'website_meta_data_snr_-25dB.csv',
 'website_meta_data_snr_0dB.csv',
 'website_meta_data_snr_-20dB.csv']

In [3]:
df = pd.read_csv(f'{meta_data_dir}/website_meta_data_snr_-30dB.csv', index_col=0)

In [3]:
meta_data_dir = 'meta_data_for_yaml'

def create_combinations(snr='-30dB', num_rows=1):
    df = pd.read_csv(f'{meta_data_dir}/website_meta_data_snr_{snr}dB.csv', index_col=0)
    #all wavs except for the input to create combinations
    #Input will be our reference
    denoised_list = list(df.iloc[0])[1:]
    #create pairs of twos
    #e.g. - (WaveUNet_SX139.WAV.n121.wav, RegressionFCNN_SX139.WAV.n121.wav) ...
    combinations = list(itertools.combinations(denoised_list, 2))
    
    # Final combination list with input appended to all 
    #e.g. - (input.wav, WaveUNet_SX139.WAV.n121.wav, RegressionFCNN_SX139.WAV.n121.wav) ...
    final_combination_list = []
    for c in combinations:
        #trick to append to tuple 
        c = (df['Input'][0], *c)
        final_combination_list.append(c)
    return final_combination_list    

In [4]:
def make_yaml_file(combinations, snr='-30dB'):
    print(f'''
    {len(combinations)} pages will be created for subjective evalutation
    ''')
    
    final_dict_template = {
    'testname': 'AB Test',
     'testId': 'ab_noloop',
     'bufferSize': 2048,
     'stopOnErrors': True,
     'showButtonPreviousPage': True,
     'remoteService': 'service/write.php',
     'pages': []
    }
    #final page which sends results
    final_page_template =  {'type': 'finish',
       'name': 'Thank you',
       'content': 'Thank you for attending',
       'showResults': False,
       'writeResults': True
    }
    pages=[]
    for i in range(len(combinations)):
        pages_dict_template = {'type': 'paired_comparison',
         'id': 'trialAB2',
         'name': None,
         'unforced': None,
         'content': 'test description AB',
         'showWaveform': True,
         'enableLooping': False,
         'reference': '',
         'stimuli': {'C1': '',
          'C2': ''}}
        pages.append(pages_dict_template)
        pages[i]['reference'] = combinations[i][0]
        pages[i]['stimuli']['C1'] = combinations[i][1]
        pages[i]['stimuli']['C2'] = combinations[i][2]
    
    pages.append(final_page_template)
    final_dict_template['pages'] = pages
    
    # dict to yaml
    yam_file_path = f'/Users/ashishalex/Documents/research/web_drone_project/webmushra/configs/snr{snr}dB.yaml'
    with open(yam_file_path, 'w') as f:
        yaml.dump(final_dict_template, f)
    print('Created yaml file at: ', yam_file_path)
    
    return final_dict_template

In [5]:
snr = '-30'

for snr in snrs:    
    combinations = create_combinations(snr=snr, num_rows=1)
    final_dict_template = make_yaml_file(combinations=combinations, snr=snr)


    10 pages will be created for subjective evalutation
    
Created yaml file at:  /Users/ashishalex/Documents/research/web_drone_project/webmushra/configs/snr0dB.yaml

    10 pages will be created for subjective evalutation
    
Created yaml file at:  /Users/ashishalex/Documents/research/web_drone_project/webmushra/configs/snr-5dB.yaml

    10 pages will be created for subjective evalutation
    
Created yaml file at:  /Users/ashishalex/Documents/research/web_drone_project/webmushra/configs/snr-10dB.yaml

    10 pages will be created for subjective evalutation
    
Created yaml file at:  /Users/ashishalex/Documents/research/web_drone_project/webmushra/configs/snr-15dB.yaml

    10 pages will be created for subjective evalutation
    
Created yaml file at:  /Users/ashishalex/Documents/research/web_drone_project/webmushra/configs/snr-20dB.yaml

    10 pages will be created for subjective evalutation
    
Created yaml file at:  /Users/ashishalex/Documents/research/web_drone_project/web